<img src="https://www.th-ab.de/typo3conf/ext/th_ab/Resources/Public/assets/logo-th-ab.svg" alt="TH-AB Logo" width="200"/>

Prof. Dr. Möckel, Prof. Dr. Radke, Katharina Kuhnert

Maschinelles Lernen Schwerpunkt Data Science<br>
SoSe 2024

# Übung 6: Erschließung eines unbekannten Datensatzes

### Bibliotheken importieren und PyTorch Umgebung prüfen

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# PyTorch
try:
    import torch
    import torchvision
    import torchmetrics
except:
    !pip install torch
    !pip install torchvision
    !pip install torchmetrics
    import torch
    import torchvision
    import torchmetrics

from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# PyTorch Version überprüfen
if (torch.__version__ < "2.0.0"):
    raise Exception("Wrong PyTorch version")
else:
    print("PyTorch Version:", torch.__version__)

PyTorch Version: 2.0.0+cpu


### Beispieldatensatz handgeschriebener Ziffern (MNIST) laden und analysieren

Der Datensatz MNIST (Modified National Institute of Standards and Technology) besteht aus handschriftlichen Ziffern (0 bis 9), die in zwei unabhängigen, aber gleich strukturierten Teilpaketen zur Verfügung gestellt werden. 

Für das Training und für alle weiteren Analysen (Validierung und Benchmarking, non-training) eines Modells stehen zum Download bereit

* Training Dataset
* Non-Training Dataset

In [3]:
# Dataset für Training
ds_train = datasets.MNIST(
    root="data", # Zielpfad für Datendownload
    train=True, # Trainingsdaten laden
    download=True,
    transform=ToTensor(), # Transformiere Features (Bilddaten) zu Tensoren
    target_transform=None # Keine Transformierung für Labels (Targets)
)

# Dataset für Validierung und Test
ds_non_train = datasets.MNIST(
    root="data",
    train=False, # Keine Trainingsdaten laden
    download=True,
    transform=ToTensor()
)

## Verständnis der geladenen Daten durch schrittweises Vorgehen:

### 1. Analyse des geladenen Datensatzes

#### 1.a) Bestimmen Sie den Datentyp der Objekte ds_train und ds_non_train und erläutern Sie das Ergebnis.

In [4]:
# Datentyp:
print("Datentyp des gesamten Trainingsdatensatzes: ", type(ds_train))
print("Datentyp des gesamten Nicht-Trainingsdatensatzes: ", type(ds_non_train))

Datentyp des gesamten Trainingsdatensatzes:  <class 'torchvision.datasets.mnist.MNIST'>
Datentyp des gesamten Nicht-Trainingsdatensatzes:  <class 'torchvision.datasets.mnist.MNIST'>


Interpretation: Es handelt sich um eine eigene Klasse.

#### 1.b) Nutzen Sie die Befehle print() und dir() um die für die Klasse des Objekts ds_train definierten Attribute und Funktionen ausgeben zu lassen:

In [5]:
print(dir(ds_train))

['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_check_exists', '_check_legacy_exist', '_format_transform_repr', '_is_protocol', '_load_data', '_load_legacy_data', '_repr_indent', 'class_to_idx', 'classes', 'data', 'download', 'extra_repr', 'mirrors', 'processed_folder', 'raw_folder', 'resources', 'root', 'target_transform', 'targets', 'test_data', 'test_file', 'test_labels', 'train', 'train_data', 'train_labels', 'training_file', 'transform', 'transforms']


#### 1.c) Betrachten im Folgenden die überladenen Standard-Funktionen für print und die Länge len sowie die Attribute "classes", "data", "train" und "train_labels"

Welche Ausgabe erfolgt durch die Print-Routine?

In [6]:
# Direkte Ausgabe des Objekts durch die print-Funktion
print(ds_train)
print(ds_non_train)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()


Bestimmen Sie den Umfang beider Teildatensätze durch die Verwendung von len und vergleichen Sie mit dem Output der Print-Routine

In [7]:
# Umfang:
print("Länge des Trainings-Datensatzes: ", len(ds_train))
print("Länge des Validierungs- und Testdatensatzes (zusammen): ", len(ds_non_train))

Länge des Trainings-Datensatzes:  60000
Länge des Validierungs- und Testdatensatzes (zusammen):  10000


Interpretation: Number of datapoints = Länge des Datensatzes

Untersuchen Sie zunächst die Attribute classes und train_labels. Was beinhalten diese? Wie ist eine Warnung zu deuten? 

In [8]:
print(ds_train.classes)

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


In [9]:
print(ds_train.train_labels)

tensor([5, 0, 4,  ..., 5, 6, 8])


c:\Users\moeckel\Anaconda3\envs\AnacondaML\lib\site-packages\torchvision\datasets\mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


Vergleichen Sie mit dem Attribut targets. Bestimmen sie für dieses seinen Datentyp, Datenformat (shape) und den Wertebereich, d.h. welche verschiedenen Werte angenommen werden können. Deuten Sie ihre Beobachtungen.

In [10]:
print(ds_train.targets)

tensor([5, 0, 4,  ..., 5, 6, 8])


In [11]:
print("Direkte Ausgabe: ", ds_train.targets)
print("Shape: ", ds_train.targets.shape)
print("Auftretende verschiedene Werte: ", ds_train.targets.unique())

Direkte Ausgabe:  tensor([5, 0, 4,  ..., 5, 6, 8])
Shape:  torch.Size([60000])
Auftretende verschiedene Werte:  tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


Interpretation: Die train_labels = targets sind die Zuordnungen zu den Klassen 0 bis 9

Was beinhaltet das Attribut train? Vergleichen Sie dazu seinen Werte auf dem training und non-training Datensatz. 

In [12]:
print(ds_train.train)

True


In [13]:
print(ds_non_train.train)

False


Interpretation: Mit dem Flag train wird entschieden, ob es sich um Trainingsdaten oder Testdaten handelt


Betrachten Sie nun das Attribut data. 
Bei der direkten Ausgabe von data ist Vorsicht ratsam (meist zu viele Daten), gehen Sie daher schrittweise vor: Lassen Sie Datentyp und Datenformat (shape) ausgeben und interpretieren Sie die Datenstruktur. Worum handelt es sich vermutlich? 

In [14]:
print(type(ds_train.data))

<class 'torch.Tensor'>


In [15]:
print(ds_train.data.shape)

torch.Size([60000, 28, 28])


Bei den Daten handelt es sich vermutlich um 60.000 sw-Bilder mit 28x28 Pixel

### 2) Analyse des ersten Elements des Datensatzes 

#### 2.a) Bestimmen Sie nun den Datentyp des ersten Elements des Datensatzes:

In [16]:
print("Datentyp des ersten Elements des Trainingsdatensatzes: ", type(ds_train[0]))
print("Datentyp des ersten Elements des Nicht-Trainingsdatensatzes: ", type(ds_non_train[0]))


Datentyp des ersten Elements des Trainingsdatensatzes:  <class 'tuple'>
Datentyp des ersten Elements des Nicht-Trainingsdatensatzes:  <class 'tuple'>


#### 2.b) Offensichtlich liegt ein Datentupel vor. Bestimmen Sie nun die Dimension des Tupels = Länge des Datentupels 

In [17]:
print(" Anzahl der Elemente im Datentupel des ersten Trainingsbeispiels: ", len(ds_train[0]))
print(" Anzahl der Elemente im Datentupel des ersten Nicht-Trainingsbeispiels: ", len(ds_non_train[0]))

 Anzahl der Elemente im Datentupel des ersten Trainingsbeispiels:  2
 Anzahl der Elemente im Datentupel des ersten Nicht-Trainingsbeispiels:  2


### 3) Zerlegung des Datentupels in Komponenten

Zerlegen Sie das erste Element des Datensatzes ds_train in zwei Elemente (hier willkürlich genannt: A,B) 

In [18]:
A,B = ds_train[0]
print("Datentyp des ersten Elements des Datentupels: ", type(A))
print("Datentyp des zweiten Elements des Datentupels: ", type(B))

Datentyp des ersten Elements des Datentupels:  <class 'torch.Tensor'>
Datentyp des zweiten Elements des Datentupels:  <class 'int'>


### 4) Analyse der Komponenten:

Bestimmen Sie jeweils deren Datentyp, ggf. Datenformat (shape) sowie falls sinnvoll die Anzahl seiner Elemente (numel) und erklären Sie so den Inhalt eines Datenpunktes

In [19]:
print("Ausgabe der Integer-Zahl: ", B, "\n")
print("Struktur des Tensors: ", A.shape)
print("Anzahl der Elemente im Tensor: ", A.numel())
print("(Zum Vergleich: 1*28*28 =", 28*28, ")")

Ausgabe der Integer-Zahl:  5 

Struktur des Tensors:  torch.Size([1, 28, 28])
Anzahl der Elemente im Tensor:  784
(Zum Vergleich: 1*28*28 = 784 )


Interpretation: Ein Datenpunkt besteht aus der Kombination von einem Bilder (28 x 28 Pixel) mit einem Label (dargestellte Ziffer). Da das Dataset für die Klassifikation von Ziffern gedacht ist entsprechen die Labels die Bezeichnungen der jeweiligen Ziffern.

### 5. Graphische Darstellung eines Beispiels aus dem Dataset:

Konvertieren Sie den Bildanteil eines Datenpunktes in eine quadratische Matrix, indem sie die redundaten Tensordimension mit squeeze reduzieren ("Tensor zusammendrücken") und geben Sie diese als Graustufenbild aus

In [1]:
plt.imshow(A.squeeze, cmap="gray")
plt.show()

NameError: name 'plt' is not defined